In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
%cd ..

In [0]:
%cd content
!ls

In [0]:
!kaggle datasets download -d nih-chest-xrays/sample -p /content
!ls

In [0]:
!unzip sample.zip

In [0]:
!mkdir images

In [0]:
import shutil
shutil.move("/content/sample.zip", "/content/images") 

In [0]:
#for i in range(10,13):
#  shutil.move("/content/images_0"+str(i)+".zip","/content/images")

In [0]:
!rm data.zip

In [0]:
!pwd

In [0]:
%cd images


In [0]:
!unzip \*.zip


In [0]:
%cd ..
!pwd

In [0]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import pandas as pd
import os
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from skimage import transform, io
from torchvision import transforms,utils
import matplotlib.pyplot as plt
from glob import glob
from itertools import chain
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
import torch.backends.cudnn as cudnn 
import torch.optim as optim


In [0]:
class ChestDataset(Dataset):
  def __init__(self, image_names, labels, transform):
    img_names = []
    for name in image_names:
      img_names.append(name)
    self.image_names = img_names
    self.labels = labels
    self.transform = transform
    
  def __getitem__(self,index):
    image_name = self.image_names[index]
    image = Image.open(image_name).convert('L')
    label = self.labels[index]
    if self.transform:
      image = self.transform(image)
    sample = {'image': image, 'label': torch.tensor(np.vstack(label).astype(np.float), dtype = torch.float32)} 
    return sample
  
  
  def __len__(self):
    return len(self.labels)

In [0]:
number_of_classes = 13
CLASS_NAMES = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
IMG_DIR = "./images/images/"
CSV_FILE = "sample_labels.csv"
batch_size = 64

In [0]:
all_xray_df = pd.read_csv(CSV_FILE)
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join(IMG_DIR, '*.png'))}
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_xray_df['Finding Labels'] = all_xray_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]

print('All Labels ({}): {}'.format(len(all_labels), all_labels))

for c_label in all_labels:
  if len(c_label)>1: # leave out empty labels
    all_xray_df[c_label] = all_xray_df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)

MIN_CASES = 20
all_labels = [c_label for c_label in all_labels if all_xray_df[c_label].sum()>MIN_CASES]
all_xray_df['disease_vec'] = all_xray_df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
print(len(all_xray_df['path']))

In [0]:
train_x,test_x,train_y, test_y = train_test_split(all_xray_df['path'].to_numpy(), all_xray_df['disease_vec'].to_numpy(),
                                   test_size = 0.40, 
                                   random_state = 2018)
test_x, val_x, test_y, val_y = train_test_split(test_x,test_y ,
                                   test_size = 0.50, 
                                   random_state = 2018)


In [0]:
test = ChestDataset(image_names = test_x, labels=test_y, 
                    transform = transforms.Compose([transforms.Scale(256), 
                    transforms.Grayscale(),transforms.ToTensor(),transforms.Normalize(mean = [0.485], std = [0.229])]))
test_loader = DataLoader(test, batch_size = 1, shuffle = True)

In [0]:
train = ChestDataset(train_x, train_y, 
                    transform = transforms.Compose([transforms.Scale(256),
                                                    transforms.Grayscale(), transforms.ToTensor(),
                                                    transforms.Normalize(mean = [0.485], std = [0.229])]))

train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)

In [0]:
valid = ChestDataset(val_x, val_y, 
                    transform = transforms.Compose([transforms.Scale(256),
                    transforms.Grayscale(), transforms.ToTensor(),
                                                    transforms.Normalize(mean = [0.485], std = [0.229])]))

valid_loader = DataLoader(valid, batch_size = batch_size, shuffle = True)

In [0]:
class Xnet(nn.Module):
  def __init__(self):
    super(Xnet,self).__init__()
    self.fc1 = nn.Linear(256*256,number_of_classes)
  def forward(self,x):
    x = x.view(-1, 256*256)
    x = F.relu(self.fc1(x))
    x = nn.Sigmoid()(x)
    return x 

In [0]:
#DEFINE THE NETWORK
class Xnet(nn.Module):
  def __init__(self):
    super(Xnet,self).__init__() 
    self.conv1 = nn.Conv2d(1,64,kernel_size = 3, padding = 1)                     #1 x 256 x 256 > 64 x 256 x 256
    self.batch_norm1 = nn.BatchNorm2d(64)                                         #BN1 = 64                                                
    self.pool = nn.MaxPool2d(2,2)                                                 # 64 x 128 x 128 
    self.conv2 = nn.Conv2d(64,128, kernel_size =3, padding =1)                    # 128 x 128 x 128
    self.batch_norm2 = nn.BatchNorm2d(128)                                        # bn2 = 128
    self.conv3 = nn.Conv2d(128,256, kernel_size = 3, padding = 1)                 # 256 x 128 x 128
    self.batch_norm3 = nn.BatchNorm2d(256) 
    self.conv4 = nn.Conv2d(256,512, kernel_size = 3, padding = 1)
    self.batch_norm4 = nn.BatchNorm2d(512) 
    self.conv5 = nn.Conv2d(512,512, kernel_size = 3, padding = 1)
    self.conv6 = nn.Conv2d(512,256, kernel_size = 3, padding = 1)
    self.conv7 = nn.Conv2d(256,128, kernel_size = 3, padding = 1)
    self.conv8 = nn.Conv2d(128,128,kernel_size = 3, padding = 1)
    self.conv9 = nn.Conv2d(128,256,kernel_size = 3, padding = 1)
    self.conv10 = nn.Conv2d(256, 512, kernel_size = 3, padding =1)
    self.conv11 = nn.Conv2d(512,512, kernel_size = 3, padding = 1)
    self.conv12 = nn.Conv2d(512,256, kernel_size = 3, padding = 1)
    self.conv13 = nn.Conv2d(256,128, kernel_size = 3, padding = 1)
    self.conv14 = nn.Conv2d(128,64, kernel_size =3, padding = 1)
    self.conv15 = nn.Conv2d(64, number_of_classes, kernel_size = 1)
    self.fc1 = nn.Linear(13*256*256, 64)
    self.fc2 = nn.Linear(64, number_of_classes)
    
  def forward(self, x):
    act1 = F.relu(self.batch_norm1(self.conv1(x)))          #64 x 256 x 256
    x = self.pool(act1)                                     #64 x 128 x 128
    act2 = F.relu(self.batch_norm2(self.conv2(x)))          #128 x 128 x 128
    x = self.pool(act2)                                     #128 x 64 x 64
    act_3 = F.relu(self.batch_norm3(self.conv3(x)))         #256 x 64 x 64
    x = self.pool(act_3)                                    #256 x 32 x 32
    x = F.relu(self.batch_norm4(self.conv4(x)))   #512 x 32 x 32
    x = F.relu(self.batch_norm4(self.conv5(x)))   #512 x 32 x 32
    x = F.upsample(x, size = 64)   #6
    x = F.relu(self.batch_norm3(self.conv6(x)))   #256 x 64 x 64    
    x = x.add(act_3)    
    x = F.upsample(x, size = 128)                 #256 x 128 x 128
    x = F.relu(self.batch_norm3(self.conv7(x)))   #128 x 128 x 128
    x = x.add(act2)    
    act_8 = F.relu(self.batch_norm2(self.conv8(x))) #128 x 128 x 128
    x = self.pool(act_8)                            #128 x 64 x 64
    act_9 = F.relu(self.batch_norm3(self.conv9(x))) #256 x 64 x 64
    x = self.pool(act_9)                            #256 x 32 x 32
    x = F.relu(self.batch_norm4(self.conv10(x)))    #512 x 32 x 32
    x = F.relu(self.batch_norm4(self.conv11(x)))    #512 x 32 x 32    
    x = F.upsample(x, size = 64)                    #512 x 64 x 64
    x = F.relu(self.batch_norm3(self.conv12(x)))    #256 x 64 x 64
    x = x.add(act_9)    
    x = F.upsample(x, size = 128)                   #256 x 128 x 128 
    x = F.relu(self.batch_norm2(self.conv13(x)))    #128 x 128 x 128
    x = x.add(act_8)    
    x = F.upsample(x, size = 256)                   #128 x 256 x 256
    x = F.relu(self.batch_norm1(self.conv14(x)))    #64 x 256 x 256
    x = x.add(act1)
    x = self.conv15(x)                              #13 x 256 x 256
    x = x.view(-1,13*256*256)
    x = F.relu(self.fc1(x))
    x = nn.Sigmoid(self.fc2(x))
    return x

In [0]:
model = Xnet()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
#criterion = nn.CrossEntropyLoss(reduction = 'sum')
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.Adam(model.parameters(), lr = 1e-4,weight_decay = 0.005)

In [0]:
EPOCHS = 10
best_acc = 0.0
best_model =  model.state_dict()

for epoch in range(EPOCHS):
  r_loss = 0
  train_correct = 0
  r_total = 0 
  model.train()
  for data in train_loader:
    
    inputs = data['image'].to(device)
    labels= data['label'].to(device).squeeze(dim = 2)
    outputs = model(inputs)
    
    optimizer.zero_grad()
    labels = labels.type(torch.FloatTensor)
    outputs =outputs.type(torch.FloatTensor)
    
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    
    outputs = torch.round(outputs)
    #print(labels.size())
    
    labels = labels.type(torch.LongTensor)
    outputs =outputs.type(torch.LongTensor)
    
    train_correct += torch.sum(outputs == labels).item()
    #print("pred")
    #print(prediction)
    #print("lab")
    #print(labels[0])
    #print(torch.sum(outputs == labels).item())
    r_loss += loss.item()
    r_total += labels.size(0)
    #print(labels.size())
  
  with torch.no_grad():
    val_losses = 0
    val_correct = 0
    model.eval()
    total = 0
    
    for data in valid_loader:
      inputs = data['image'].to(device)
      labels= data['label'].to(device).squeeze(dim = 2)
      outputs = model(inputs)
      
      labels = labels.type(torch.FloatTensor)
      outputs =outputs.type(torch.FloatTensor)
      
      val_loss = criterion(outputs,labels)
      val_losses += val_loss.item()
      
      outputs = torch.round(outputs)
      
      labels = labels.type(torch.LongTensor)
      outputs =outputs.type(torch.LongTensor)
      val_correct += torch.sum(outputs ==labels).item()
      total += labels.size(0)
      
  r_loss = r_loss/total
  train_correct = train_correct/r_total/number_of_classes
  
  val_loss = val_losses/ total
  val_correct = val_correct/ total/number_of_classes
  
  if val_correct > best_acc:
    bect_acc = val_correct
    best_model = model.state_dict()
  
       
  print("Epoch:" + str(epoch) +". Training loss:{: .3f}.  Train correct: {: .3f}".format(r_loss, train_correct))
  print('Validation loss: {: .3f}, {: .3f}'.format(val_loss, val_correct))

In [0]:
#!mkdir models
torch.save(best_model, "./models/model" + str(20) + ".pth")

In [0]:
#Testing process
with torch.no_grad():
  correct = 0
  total = 0
  for data in test_loader:
    inputs = data['image'].to(device)
    labels= data['label'].to(device).squeeze(dim = 2)
    outputs = model(inputs)
    outputs = torch.round(outputs)
    
    labels = labels.type(torch.LongTensor)
    outputs =outputs.type(torch.LongTensor)
    
    total +=labels.size(0)
    correct +=torch.sum(outputs == labels).item()
print("Accuracy of the network is %f %%" %(correct*100/number_of_classes/ total))
    
    
